In [1]:
import numpy as np
import pandas as pd
from roar_py_carla.carla_agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
dat = pd.read_csv("new_right_side_waypoints.txt", header=None)

In [3]:
print(dat)

                 0           1            2
0      2599.199951  104.746101  4372.600098
1      2599.199951  104.682404  4372.600098
2      2599.199951  104.594200  4372.600098
3      2599.199951  104.481499  4372.600098
4      2599.199951  104.344292  4372.600098
...            ...         ...          ...
46063  2603.321289  107.629265  4270.680664
46064  2603.321289  107.628433  4270.694824
46065  2603.321289  107.627632  4270.708496
46066  2603.321289  107.626801  4270.722656
46067  2603.321289  107.625954  4270.736328

[46068 rows x 3 columns]


In [10]:
dat_np = np.asarray(dat)

In [11]:
dat_np
print(dat_np.shape)
dat_np = dat_np[:,[0,2,1]]
dat_np_sparse = dat_np[::50,:]
print(dat_np_sparse.shape)

(46068, 3)
(922, 3)


In [9]:
dat_np_sparse

array([[2599.19995117, 4372.60009766,  104.74610138],
       [2599.19995117, 4372.60009766,  103.00659943],
       [2599.19995117, 4372.59179688,  102.78607178],
       ...,
       [2603.32104492, 4270.36279297,  107.64844513],
       [2603.32128906, 4270.49853516,  107.64022064],
       [2603.32128906, 4270.63867188,  107.63179779]])

In [15]:
import carla
import roar_py_carla
import roar_py_interface
import networkx as nx

carla_client = carla.Client('localhost', 2000)
carla_client.set_timeout(5.0)
roar_py_instance = roar_py_carla.RoarPyCarlaInstance(carla_client)
roar_py_world = roar_py_instance.world
grp = GlobalRoutePlanner(roar_py_world._native_carla_map, 2.0)
native_ws = []


for i in range(len(dat_np_sparse)):
    curr_start = dat_np_sparse[i]
    curr_end = dat_np_sparse[(i+1)%len(dat_np_sparse)]
    loc_1 = carla.Location(curr_start[0], curr_start[1], curr_start[2])
    loc_2 = carla.Location(curr_end[0], curr_end[1], curr_end[2])
    try:
        native_ws += grp.trace_route(loc_1, loc_2)
    except nx.NetworkXNoPath:
        continue

In [16]:
real_ws = []
for native_ww in native_ws:
    w = native_ww[0]
    transform_w = roar_py_carla.transform_from_carla(w.transform)
    transform_w[1][2] += np.pi/2
    real_w = roar_py_interface.RoarPyWaypoint(
        transform_w[0],
        transform_w[1],
        w.lane_width
    )
    real_ws.append(real_w)

In [17]:
np.savez_compressed("final_major_map_waypoints.npz", **roar_py_interface.RoarPyWaypoint.save_waypoint_list(real_ws))

In [28]:
from roar_py_interface import RoarPyWaypoint
from typing import List, Tuple
from roar_py_carla.utils import *
import carla

def generate_waypoints(locations : List[np.ndarray], distance_between_waypoints : float, lane_width : float):
    locations = [location_from_carla(carla.Location(x=location[0], y=location[1], z=location[2])) for location in locations]
    waypoint_locations = [locations[0]]
    waypoint_rotations = []
    last_location_idx = 0
    for i in range(1,len(locations)):
        next_location = locations[i]
        dist_to_last_location = np.linalg.norm(next_location - locations[last_location_idx])
        if dist_to_last_location > distance_between_waypoints:
            waypoint_locations.append(next_location)
            waypoint_rotations.append(np.arctan2(next_location[1] - locations[last_location_idx][1], next_location[0] - locations[last_location_idx][0]))
            last_location_idx = i
    waypoint_rotations.append(np.arctan2(locations[0][1] - waypoint_locations[-1][1], locations[0][0] - waypoint_locations[-1][0]))
    waypoints = []
    for i in range(len(waypoint_locations)):
        waypoints.append(RoarPyWaypoint(
            waypoint_locations[i],
            np.array([0,0,waypoint_rotations[i]]),
            lane_width
        ))
    return waypoints

waypoints = generate_waypoints(dat_np, 2, 3.5)
print("final length",len(waypoints))

np.savez_compressed("final_major_map_waypoints.npz", **RoarPyWaypoint.save_waypoint_list(waypoints))
print(waypoints[0].location)

final length 6942
[-4372.60009766 -2599.19995117   104.74610138]


In [26]:

def generate_spawn_points(waypoints : List[RoarPyWaypoint]) -> Tuple[np.ndarray, np.ndarray]:
    spawn_point_locations = []
    spawn_rotations = []
    for spawn_point in waypoints[::len(waypoints)//10]:
        spawn_point_locations.append(spawn_point.location)
        spawn_rotations.append(spawn_point.roll_pitch_yaw)
    return np.stack(spawn_point_locations, axis=0), np.stack(spawn_rotations, axis=0)

spawn_point_locations, spawn_rotations = generate_spawn_points(waypoints)
print(spawn_point_locations.shape)
np.savez_compressed("final_major_map_spawn_points.npz", locations=spawn_point_locations, rotations=spawn_rotations)

(11, 3)
